In [1]:
import numpy as np
import tnn
import torch
import torch.nn as nn
import torch.utils.data as data
import logging

from datasets import load_dataset, concatenate_datasets
from transformers import BertTokenizer, BertModel

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

In [2]:
dataset = load_dataset("dair-ai/emotion", "split")
train = dataset.get("train")
val = dataset.get("validation")
test = dataset.get("test")
eval = concatenate_datasets([val, test])

In [3]:
classes = len(train.unique("label"))
name = "google-bert/bert-large-uncased"
tokenizer = BertTokenizer.from_pretrained(name)
model = tnn.BertForClassification(classes=classes, hidden_size=1024, name=name)

/home/andrew/git/teaching-neural-networks/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
def pre_process(examples):
    examples = tokenizer(
        examples["text"],
        max_length=128,
        truncation=True,
        padding="max_length",
    )
    return examples


train_dataset = train.map(
    pre_process, batch_size=True, num_proc=2, remove_columns=["text"]
)
eval_dataset = eval.map(
    pre_process, batch_size=True, num_proc=2, remove_columns=["text"]
)

In [5]:
def collate_fn(batch):
    inputs = {"input_ids": [], "token_type_ids": [], "attention_mask": []}
    labels = []

    for example in batch:
        inputs["input_ids"].append(example["input_ids"])
        inputs["token_type_ids"].append(example["token_type_ids"])
        inputs["attention_mask"].append(example["attention_mask"])
        labels.append(example["label"])

    inputs["input_ids"] = torch.tensor(inputs["input_ids"]).long()
    inputs["token_type_ids"] = torch.tensor(inputs["token_type_ids"]).long()
    inputs["attention_mask"] = torch.tensor(inputs["attention_mask"]).long()
    labels = torch.tensor(labels).long()
    return inputs, labels

In [7]:
trainlaoder = data.DataLoader(
    train_dataset,
    batch_size=32,
    collate_fn=collate_fn,
    num_workers=2,
    pin_memory=True,
    drop_last=False,
    shuffle=True,
)

testloader = data.DataLoader(
    eval_dataset,
    batch_size=32,
    collate_fn=collate_fn,
    num_workers=True,
    pin_memory=True,
    drop_last=False,
    shuffle=False,
)

for inputs, labels in trainlaoder:
    outputs = model(**inputs)
    print(outputs.get("logits").size())
    break
    

torch.Size([32, 6])


KeyboardInterrupt: 